In [ ]:
# @title Setup
!pip install looker_sdk
!pip install google-cloud-aiplatform --upgrade --user
!pip install pandas

from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
import looker_sdk
import pandas as pd
import urllib3
import os
import yaml
import string
import random
import json
import time
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

os.environ['LOOKERSDK_BASE_URL'] = 'https://demo.looker.com'
os.environ['LOOKERSDK_CLIENT_ID'] = 'xx'
os.environ['LOOKERSDK_CLIENT_SECRET'] = 'xxx'
os.environ['LOOKERSDK_VERIFY_SSL']= 'False'
os.environ['LOOKERSDK_API_VERSION']= '4.0'
os.environ["LOOKERSDK_TIMEOUT"] = "120"

sdk = looker_sdk.init40()

## Generating a data dictionnary from Looker Instance
Add description later to this

In [ ]:
def get_field_values(model_name, explore_name):

    sdk = looker_sdk.init40()

    # API Call to pull in metadata about fields in a particular explore
    explore = sdk.lookml_model_explore(
        lookml_model_name=model_name,
        explore_name=explore_name,
        fields="id, name, description, fields, label",
    )

    measures = []
    dimensions = []

    # Iterate through the field definitions and pull in the description, sql,
    # and other looker tags you might want to include in  your data dictionary.
    if explore.fields and explore.fields.dimensions:
        for dimension in explore.fields.dimensions:
          def_dimension = {
              "name": dimension.name,
              "description": dimension.description,
          }
          dimensions.append(def_dimension)

    if explore.fields and explore.fields.measures:
        for measure in explore.fields.measures:
          def_measure = {
             "name": measure.name,
             "description": measure.description,
          }
          measures.append(def_measure)


    #print(dimensions)
    #print(measures)
    # Create a dataframe of the dimensions and metrics

    #df = pd.concat([dimensions, measures])

    return dimensions, measures

# Getting and formatting the results correctly
Add description later


In [ ]:
results_model = get_field_values("thelook", "order_items") # the model and the explore name.
dimensions = json.dumps(results_model[0])
measures = json.dumps(results_model[1])
#print(dimensions) to see the results
#print(measures)

[{"name": "discounts.date_date", "description": ""}, {"name": "discounts.date_day_of_month", "description": ""}, {"name": "discounts.date_day_of_week", "description": ""}, {"name": "discounts.date_day_of_week_index", "description": ""}, {"name": "discounts.date_day_of_year", "description": ""}, {"name": "discounts.date_hour", "description": ""}, {"name": "discounts.date_hour_of_day", "description": ""}, {"name": "discounts.date_minute", "description": ""}, {"name": "discounts.date_month", "description": ""}, {"name": "discounts.date_month_name", "description": ""}, {"name": "discounts.date_month_num", "description": ""}, {"name": "discounts.date_quarter", "description": ""}, {"name": "discounts.date_quarter_of_year", "description": ""}, {"name": "discounts.date_raw", "description": ""}, {"name": "discounts.date_time", "description": ""}, {"name": "discounts.date_time_of_day", "description": ""}, {"name": "discounts.date_week", "description": ""}, {"name": "discounts.date_week_of_year",

## Inititating Text Bison to be used
Add description later

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel

vertexai.init(project="data-analytics-386814", location="us-central1")
parameters = {
    "temperature": 0.1,
    "max_output_tokens": 1024,
    "top_p": 0.8,
    "top_k": 40
}
model = TextGenerationModel.from_pretrained("text-bison")

# Iterating on PALM 2 API to describe empty fields
Add description later


In [ ]:
data = json.loads(measures)
result_string = ""

content = """You role is to give a short description (maximum 30 charachters) for the following database field, in the context of e-commerce

""" ### Customize this prompt as you wish.

for d in data:
  field = d['name']
  description = model.predict(content + field,
     **parameters
  )
  print(f"{field}:{description}\n")
  result_string += f"{field}:{description}\n"
  time.sleep(1)

turtle::high_value_geos:A list of geographic areas that are high-value for the business.

turtle::inventory_aging:The number of days that a product has been in inventory.

turtle::severely_delayed_orders:Orders that have been delayed by more than 3 days.

turtle::shipments_status:The status of a shipment, e.g. "shipped", "in transit", "delivered".

turtle::year_over_year:_growth Year-over-year growth in sales for a given product or category.

discounts.average_discount:The average discount offered by a store on its products.

discounts.count:Number of discounts available for a product.

distribution_centers.location_latitude_max:The maximum latitude of the distribution center's location.

distribution_centers.location_latitude_min:The minimum latitude of the distribution center's location.

distribution_centers.location_longitude_max:The maximum longitude of all distribution centers.

distribution_centers.location_longitude_min:The minimum longitude of the distribution center's locatio

# Formatting in a simple way
Add description later

In [ ]:
print(result_string)

turtle::high_value_geos:A list of geographic areas that are high-value for the business.
turtle::inventory_aging:The number of days that a product has been in inventory.
turtle::severely_delayed_orders:Orders that have been delayed by more than 3 days.
turtle::shipments_status:The status of a shipment, e.g. "shipped", "in transit", "delivered".
turtle::year_over_year:_growth Year-over-year growth in sales for a given product or category.
discounts.average_discount:The average discount offered by a store on its products.
discounts.count:Number of discounts available for a product.
distribution_centers.location_latitude_max:The maximum latitude of the distribution center's location.
distribution_centers.location_latitude_min:The minimum latitude of the distribution center's location.
distribution_centers.location_longitude_max:The maximum longitude of all distribution centers.
distribution_centers.location_longitude_min:The minimum longitude of the distribution center's location.
invento